In [1]:
import sys

!{sys.executable} -m pip install farm-haystack -f https://download.pytorch.org/whl/torch_stable.html
!{sys.executable} -m pip install farm-haystack[ocr]

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached farm_haystack-1.9.1-py3-none-any.whl (733 kB)
  Using cached elasticsearch-7.10.1-py2.py3-none-any.whl (322 kB)
  Using cached more_itertools-8.14.0-py3-none-any.whl (52 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached python-docx-0.8.11.tar.gz (5.6 MB)
  Using cached tika-1.24.tar.gz (28 kB)
  Using cached quantulum3-0.7.10-py3-none-any.whl (10.7 MB)
  Using cached elastic-apm-6.12.0.tar.gz (174 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached mlflow-1.29.0-py3-none-any.whl (16.9 MB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Using cached sentence-t

ERROR: Invalid requirement: "'farm-haystack[ocr]'"


In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [2]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [4]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "Datasets/Watchman"

#s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip"
#fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "<some-document-name>", "content": "<the-actual-text>"}

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the docs to our DB.
document_store.write_documents(docs)

INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\Alternative to Warfarin – WATCHMAN Implant.pdf
INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\How The WATCHMAN Device Works.pdf
INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\WATCHMAN Implant for Non-Valvular Afib Stroke Risk.pdf
INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\WATCHMAN Procedure vs. Blood Thinners.pdf


[<Document: {'content': "Alternative to Blood Thinners\nTake a short survey to see if an implant could be\nthe right stroke-risk reduction option for you\nStart Here\nThere are different treatments available to reduce your stroke risk if you have AFib. The most common are blood\nthinners. Learn about blood thinner alternatives below.\nIs There an Alternative to Blood Thinners?\nBlood thinner medications, also called anticoagulant drugs, are an effective way to lower the risk of stroke in\npeople with atrial fibrillation, or AFib, not caused by a heart valve problem.\nThe blood thinner warfarin (also known as Coumadin®) has been around for more than 60 years. There are also\nseveral newer blood thinners available now, including Eliquis® (apixaban), Pradaxa® (dabigatran),\nXarelto® (rivaroxaban), and Savaysa® (edoxaban).\nFor people who need an alternative, there are heart procedures that can effectively reduce the risk of stroke.\nWATCHMAN is a permanent implant that does just that.\nLe

In [5]:
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO - haystack.nodes.retriever.sparse -  Found 4 candidate paragraphs from 4 docs in DB


In [6]:
from haystack.nodes import FARMReader


# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 11 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\   /w\   /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \   /'\   /'\   / \   / \   /'\   /'\   /'\   /'\   /'\ 


In [16]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [26]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="What are the risks of a Watchman implant procedure?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.05 Batches/s]


In [27]:
# Now you can either print the object directly...
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'stroke risk and bleeding worry for life', 'type': 'extractive', 'score': 0.5447235703468323, 'context': 's a one‑time, minimally invasive procedure that\nreduces stroke risk and bleeding worry for life.\nOne Time. For a Lifetime.\nAND COUNTING\nLIVES CHANGED\n', 'offsets_in_document': [{'start': 314, 'end': 353}], 'offsets_in_context': [{'start': 56, 'end': 95}], 'document_id': '3009c1b472f32065d7c145ed1a56e421', 'meta': {'name': 'WATCHMAN Implant for Non-Valvular Afib Stroke Risk.pdf'}}>,
             <Answer {'answer': 'stroke', 'type': 'extractive', 'score': 0.4704113006591797, 'context': 'by the U.S. Food and Drug Administration (FDA) for reducing\nthe risk of stroke in people with atrial fibrillation not caused by a heart\nvalve problem.', 'offsets_in_document': [{'start': 4345, 'end': 4351}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_id': '69f555cef7b09873f4d75f030f68a300', 'meta': {'name': 'How The WATCHMAN Device Works.pdf'}}>,

In [28]:
# ...or use a util to simplify the output
from haystack.utils import print_answers


# Change `minimum` to `medium` or `all` to control the level of detail
print_answers(prediction, details="minimum")


Query: What are the risks of a Watchman implant procedure?
Answers:
[   {   'answer': 'stroke risk and bleeding worry for life',
        'context': 's a one‑time, minimally invasive procedure that\n'
                   'reduces stroke risk and bleeding worry for life.\n'
                   'One Time. For a Lifetime.\n'
                   'AND COUNTING\n'
                   'LIVES CHANGED\n'},
    {   'answer': 'stroke',
        'context': 'by the U.S. Food and Drug Administration (FDA) for '
                   'reducing\n'
                   'the risk of stroke in people with atrial fibrillation not '
                   'caused by a heart\n'
                   'valve problem.'},
    {   'answer': 'people with atrial fibrillation not caused by a heart valve '
                  'problem who need an\n'
                  'alternative to blood thinners',
        'context': 'CHMAN Implant is for people with atrial fibrillation not '
                   'caused by a heart valve problem who ne